# DEF Notebook
This notebook aims to extend the work of the ABC notebook into a Dynamics & Environment Focus (DEF), where the initial ABCs will be turned into movies outlining the evolution of all statistics. The evolutions will both be through time and environment, each showcasing the unique role that phases play in the Large Scale Structure of the Universe.

# Data Import

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as spy
import os
import scipy.optimize as spyfit

def DataImport(filedir,nx,ny,nz,lx,ly,lz):
    print('Processing amplitudes and phases of file:')
    print(filedir)
    data = np.loadtxt(filedir)
    x_data, y_data, z_data = data[:,1], data[:,2], data[:,3]
    datagrid = np.zeros((nx,ny,nz)) #initialize the data grid
    datagrid, edges = np.histogramdd(np.vstack([x_data,y_data,z_data]).transpose(),bins=(nx,ny,nz),range=((0.,lx),(0.,ly),(0.,lz)))
    mean_density = np.float(len(x_data))/np.float(nx*ny*nz)
    overdensities = datagrid/mean_density-1 +0j
    FFTfield = np.fft.fftn(overdensities) #Perform a Fourier Transform on the density field
    amplitudes = np.sqrt(FFTfield.real**2 + FFTfield.imag**2)
    phases = np.arctan2(FFTfield.imag, FFTfield.real)
    print('Done!')
    return overdensities, mean_density, amplitudes, phases

In [40]:
def PowerSpectra(mean_density, resolution, amps, phases, k_tot, redshift):
    print('Plotting Power Spectra... ({0})'.format(redshift))
    binnum = 30 #Number of total bins
    kmin = np.min(k_tot)
    kmax = np.max(k_tot)
    dk = (kmax-kmin)/binnum #Width of a bin
    ikbin = np.digitize(k_tot,np.linspace(kmin,kmax,binnum+1)) #Bin the values of k total into binnum
    nmodes,pk,pk_err = np.zeros(binnum,dtype=int),np.full(binnum,-1.),np.full(binnum,-1.) #initialize nmodes and power spectrum
    amp_pwr = amps**2
    phase_pwr = phases**2
    binmids=np.zeros(binnum)
    binmids = np.linspace(kmin+0.5*dk, kmax-0.5*dk, binnum)

    shotnoise = resolution**3/mean_density
    phasenoise = np.pi**2/3

    #Initialize other power spectra arrays
    phk, phk_err = np.full(binnum,-1.),np.full(binnum,-1.)
    for ik in range(binnum):
        nmodes[ik] = int(np.sum(np.array([ikbin == ik+1])))
        if (nmodes[ik] > 0):
            #GiggleZ Amp power spectrum
            pk[ik] = np.mean(amp_pwr[ikbin == ik+1])
            pk_err[ik] = np.std(amp_pwr[ikbin == ik+1])/np.sqrt(nmodes[ik]/2)
            #GiggleZ Phase power spectrum
            phk[ik] = np.mean(phase_pwr[ikbin == ik+1])
            phk_err[ik] = np.std(phase_pwr[ikbin == ik+1])/np.sqrt(nmodes[ik]/2)

    plt.close("all")

    plt.figure('AmpPwr')
    plt.title('Amplitude Power Spectra ({0}) (Noise Reduced)'.format(redshift))
    plt.xlabel('$k_{total}$')
    plt.ylabel('Amplitude Power')
    plt.ylim([0,1e8])
    plt.scatter(binmids, (pk-shotnoise)*binmids**2, s=8, color='blue')
    plt.errorbar(binmids,(pk-shotnoise)*binmids**2,pk_err,color='blue')
    plt.savefig('C:/Users/Simon/Documents/Honours/Phases of the Universe/AmpPwr/f_{0}.png'.format(redshift), bbox_inches='tight')

    plt.figure('PhasePwr')
    plt.title('Phase Power Spectra ({0}) (Noise Reduced)'.format(redshift))
    plt.xlabel('$k_{total}$')
    plt.ylabel('Phase Power')
    plt.ylim([-0.5, 0.5])
    plt.scatter(binmids, phk-phasenoise, s=8, color='blue')
    plt.errorbar(binmids,phk-phasenoise,phk_err,color='blue')
    plt.savefig('C:/Users/Simon/Documents/Honours/Phases of the Universe/PhasePwr/f_{0}.png'.format(redshift), bbox_inches='tight')

    print('Done!')
    
def InformationEntropy(Sphases, nx, ny, nz, lx):
    SD_k = np.roll(Sphases,1,axis=0)-Sphases
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if SD_k[i,j,k] < -np.pi:
                    SD_k[i,j,k] += 2*np.pi
                if SD_k[i,j,k] > np.pi:
                    SD_k[i,j,k] -= 2*np.pi
                
    Sdist = SD_k[SD_k != 0].flatten()
    S_freq_dist, null = np.histogram(Sdist)
    SS_D = -np.sum(S_freq_dist*np.log(S_freq_dist)*(lx/nx))
    print('Information Entropy: {0}'.format(SS_D))
    return SS_D

def theo_func(x_data, a):
    return 1/(2*np.pi)+a*np.cos(x_data)
    
def getphasesum(densgrid,kmin,kmax,nx,ny,nz,lx,ly,lz,redshift):
# FFT density grid
    densspec = np.fft.fftn(densgrid)
# Obtain grid of phases
    phasegrid = np.arctan2(np.imag(densspec),np.real(densspec))
# Determine wavenumber at each grid point
    kx = 2.*np.pi*np.fft.fftfreq(nx,d=lx/nx)
    ky = 2.*np.pi*np.fft.fftfreq(ny,d=ly/ny)
    kz = 2.*np.pi*np.fft.fftfreq(nz,d=lz/nz)
    kgrid = np.sqrt(kx[:,np.newaxis,np.newaxis]**2 + ky[np.newaxis,:,np.newaxis]**2 + kz[np.newaxis,np.newaxis,:]**2)
# (x,y,z) mode indices corresponding to each grid point
    ikxgrid = np.transpose(np.tile(np.arange(nx),(nz,ny,1)),(2,1,0))
    ikygrid = np.transpose(np.tile(np.arange(ny),(nx,nz,1)),(0,2,1))
    ikzgrid = np.transpose(np.tile(np.arange(nz),(ny,nx,1)),(1,0,2))
# This is a (true,false) grid of whether each mode is independent
    indep = getindep(nx,ny,nz)
# Determine list of modes lying in wavenumber range
    cut = (kgrid > kmin) & (kgrid < kmax)
    ikxlst,ikylst,ikzlst,indep = ikxgrid[cut],ikygrid[cut],ikzgrid[cut],indep[cut]
    nmodes = len(ikxlst)
# Just use the independent modes for k1
    ikxlst1,ikylst1,ikzlst1 = ikxlst[indep],ikylst[indep],ikzlst[indep]
    nmodes1 = len(ikxlst1)
    print (nmodes1,'modes in bin)')
# Loop over first mode k1
    phasesumlst = []
    for i in range(nmodes1):
        ix1,iy1,iz1 = ikxlst1[i],ikylst1[i],ikzlst1[i]
        if (np.mod(i,100) == 1):
            print ('Analyzing mode',i+1,'/',nmodes1)
# Loop over second mode k2
    for j in range(nmodes):
        ix2,iy2,iz2 = ikxlst[j],ikylst[j],ikzlst[j]
# Determine third mode k3 to close triangle
        ix3,iy3,iz3 = getk3(nx,ny,nz,ix1,iy1,iz1,ix2,iy2,iz2)
# Add up the phases of these modes
        phasesum = phasegrid[ix1,iy1,iz1] + phasegrid[ix2,iy2,iz2] + phasegrid[ix3,iy3,iz3]
# Map each phase sum into the [0,2*pi] interval
        phasesum = np.mod(phasesum,2.*np.pi)
# Add each phase sum to the list
        phasesumlst.append(phasesum)
    phasesumlst = np.array(phasesumlst)
    print (len(phasesumlst),'phase sums measured')

    pshist, null = np.histogramdd(phasesumlst, normed=True, bins=100)
    X = np.linspace(0,2*np.pi,len(pshist))
    a_val, null = spyfit.curve_fit(theo_func, X, pshist)
    pshist_theo = theo_func(X, a_val)

    plt.close("all")
    plt.figure()
    plt.scatter(X, pshist, color='blue', label='Sample Phase Sums', alpha=0.4, s=4)
    plt.plot([0,2*np.pi],[1/(2*np.pi),1/(2*np.pi)], color='red', label='GRF fit')
    plt.plot(X, pshist_theo, label='Sample Fit $a = {0}$'.format(a_val))
    plt.xlim([0,2*np.pi])
    plt.ylim([0.145,0.175])
    plt.title('Phase Sum Distribution for ${0}<k<{1}$ ({2})'.format(kmin, kmax, redshift))
    plt.xlabel('Phase Sum $\phi(k_1) + \phi(k_2) + \phi(-k_1-k_2)$')
    plt.ylabel('Distribution')
    plt.legend()
    plt.savefig('C:/Users/Simon/Documents/Honours/Phases of the Universe/PhaseSums/f_{0}.png'.format(redshift), bbox_inches='tight')

# Determine third mode k3 to close triangle
def getk3(nx,ny,nz,ix1,iy1,iz1,ix2,iy2,iz2):
    ix3 = -(ix1+ix2)
    iy3 = -(iy1+iy2)
    iz3 = -(iz1+iz2)
# Map integers back into the [0,n] interval if required
    if (ix3 < 0):
        ix3 += nx
    if (iy3 < 0):
        iy3 += ny
    if (iz3 < 0):
        iz3 += nz
    return ix3,iy3,iz3

# This is a (true,false) grid of whether each mode is independent
def getindep(nx,ny,nz):
    indep = np.full((nx,ny,nz),False,dtype=bool)
    indep[:,:,1:int(nz/2)] = True
    indep[1:int(nx/2),:,0] = True
    indep[1:int(nx/2),:,int(nz/2)] = True
    indep[0,1:int(ny/2),0] = True
    indep[0,1:int(ny/2),int(nz/2)] = True
    indep[int(nx/2),1:int(ny/2),0] = True
    indep[int(nx/2),1:int(ny/2),int(nz/2)] = True
    indep[int(nx/2),0,0] = True
    indep[0,int(ny/2),0] = True
    indep[int(nx/2),int(ny/2),0] = True
    indep[0,0,int(nz/2)] = True
    indep[int(nx/2),0,int(nz/2)] = True
    indep[0,int(ny/2),int(nz/2)] = True
    indep[int(nx/2),int(ny/2),int(nz/2)] = True
    return indep

def Main():
    resolution = 256 #Dictates the number of bins in x,y,z (Use powers of 2)
    field_length = 1000 #The length of x,y,z in Mpc/h
    nx,ny,nz = resolution,resolution,resolution #number of bins in x, y and z. Stick to powers of 2!
    lx,ly,lz = field_length,field_length,field_length #needs units (Mpc/h)! #length of each dimension
    cwd = os.getcwd()
    
    dx,dy,dz = lx/nx,ly/ny,lz/nz #length of each bin
    kx = 2.*np.pi*np.fft.fftfreq(nx,d=lx/nx) 
    ky = 2.*np.pi*np.fft.fftfreq(ny,d=ly/ny)
    kz = 2.*np.pi*np.fft.fftfreq(nz,d=lz/nz)
    k_tot = np.sqrt(kx[:,np.newaxis,np.newaxis]**2 + ky[np.newaxis,:,np.newaxis]**2 + kz[np.newaxis,np.newaxis,:]**2)
    
    datasets = ['z0pt000.dat', 'z0pt208.dat', 'z0pt299.dat', 'z0pt408.dat',
           'z0pt509.dat', 'z0pt593.dat', 'z0pt687.dat', 'z0pt721.dat',
           'z0pt791.dat', 'z0pt989.dat', 'z1pt224.dat', 'z1pt386.dat',
           'z1pt504.dat', 'z2pt070.dat']
    
    redshifts = ['z=0.000', 'z=0.208', 'z=0.299', 'z=0.408', 
                 'z=0.509', 'z=0.593', 'z=0.687', 'z=0.721', 
                 'z=0.791', 'z=0.989', 'z=1.224', 'z=1.386', 'z=1.504', 'z=2.070']
    
    IEs = np.zeros(len(datasets))
    for dataset in range(len(datasets)):
        ODs, mean_dens, amps, phases = 0, 0, 0, 0
        ODs, mean_dens, amps, phases = DataImport('{0}/GiggleZ redshift data/{1}'.format(cwd,datasets[dataset]),nx,ny,nz,lx,ly,lz) #Returns amplitudes and phases of dataset. Also saves overdensity field image.
        #PowerSpectra(mean_dens, resolution, amps, phases, k_tot, redshifts[dataset]) #Creates Power spectra of current dataset and saves comparative plots
        #IEs[dataset] = InformationEntropy(phases,nx,ny,nz,lx) #Prints and returns the value of Information Entropy for the given sample
        k_low = 0.01
        k_hi = 0.1
        getphasesum(ODs, k_low, k_hi, nx, ny, nz, lx, ly, lz, redshifts[dataset])
    #[for loop ends]
    #zvals = [0.,0.208,0.299,0.408,0.509,0.593,0.687,0.721,0.791,0.989,1.224,1.386,1.504,2.07]
    #plt.figure('Information Entropies across redshift')
    #plt.scatter(zvals,IEs)
    #plt.plot(zvals,IEs)
    #plt.xlabel('Redshift (z)')
    #plt.ylabel('Information Entropy')
    #plt.savefig('C:/Users/Simon/Documents/Honours/Phases of the Universe/plots/IE_redshift.png')

    #agevals = [13.721,11.185,10.282,9.333,8.563,7.993,7.42,7.228,6.855,5.948,5.094,4.613,4.306,3.207]
    #plt.figure('Information Entropies over age (Gyr)')
    #plt.scatter(agevals,IEs)
    #plt.plot(agevals,IEs)
    #plt.xlabel('Universe Age (Gyr)')
    #plt.ylabel('Information Entropy')
    #plt.savefig('C:/Users/Simon/Documents/Honours/Phases of the Universe/plots/IE_age.png')
    #MoVis()

IndentationError: unindent does not match any outer indentation level (<ipython-input-40-e3c6f9403295>, line 71)

In [38]:
Main()

Processing amplitudes and phases of file:
C:\Users\Simon\Documents\Honours\Phases of the Universe\Jupyter notebook/GiggleZ redshift data/z0pt000.dat
Done!
8430 modes in bin)
Analyzing mode 2 / 8430
Analyzing mode 102 / 8430
Analyzing mode 202 / 8430
Analyzing mode 302 / 8430
Analyzing mode 402 / 8430
Analyzing mode 502 / 8430
Analyzing mode 602 / 8430
Analyzing mode 702 / 8430
Analyzing mode 802 / 8430
Analyzing mode 902 / 8430
Analyzing mode 1002 / 8430
Analyzing mode 1102 / 8430
Analyzing mode 1202 / 8430
Analyzing mode 1302 / 8430
Analyzing mode 1402 / 8430
Analyzing mode 1502 / 8430
Analyzing mode 1602 / 8430
Analyzing mode 1702 / 8430
Analyzing mode 1802 / 8430
Analyzing mode 1902 / 8430
Analyzing mode 2002 / 8430
Analyzing mode 2102 / 8430
Analyzing mode 2202 / 8430
Analyzing mode 2302 / 8430
Analyzing mode 2402 / 8430
Analyzing mode 2502 / 8430
Analyzing mode 2602 / 8430
Analyzing mode 2702 / 8430
Analyzing mode 2802 / 8430
Analyzing mode 2902 / 8430
Analyzing mode 3002 / 8430
A

KeyboardInterrupt: 